In [21]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import hashlib


# Working with Images Lab
## Information retrieval, preprocessing, and feature extraction

In this lab, you'll work with images of felines (cats), which have been classified according to their taxonomy. Each subfolder contains images of a particular species. The dataset is located [here](https://www.kaggle.com/datasets/datahmifitb/felis-taxonomy-image-classification) but it's also provided to you in the `data/` folder.

### Problem 1. Some exploration (1 point)
How many types of cats are there? How many images do we have of each? What is a typical image size? Are there any outliers in size?

In [12]:
file_name = os.listdir('data/')
cat_types = len(file_name)
cat_types

7

In [13]:
image_count_cats = {}
for type in file_name:
    current_type = os.path.join('data/', type)
    file_name_current = os.listdir(current_type)
    current_image_number = len(file_name_current)
    image_count_cats[type] = current_image_number

image_count_cats


{'african-wildcat': 91,
 'blackfoot-cat': 79,
 'chinese-mountain-cat': 43,
 'domestic-cat': 64,
 'european-wildcat': 85,
 'jungle-cat': 86,
 'sand-cat': 73}

In [14]:
file_stats = os.stat('data/')
img = mpimg.imread('data/african-wildcat/af (1).jpg')
img_shape = img.shape
img_shape

(190, 265, 3)

### Problem 2. Duplicat(e)s (1 point)
Find a way to filter out (remove) identical images. I would recommnend using file hashes, but there are many approaches. Keep in mind that during file saving, recompression, etc., a lot of artifacts can change the file content (bytes), but not visually.

In [20]:
def calculate_hash(file_path, hash_function=hashlib.md5):
    """Calculate the hash of a file."""
    hasher = hash_function()
    with open(file_path, 'rb') as file:
        buf = file.read()
        hasher.update(buf)
    return hasher.hexdigest()

def remove_duplicate_images(image_folder):
    """Remove duplicate images in the specified folder."""
    hashes = {}
    for root, dirs, files in os.walk(image_folder):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # Calculate the hash of the image file
            file_hash = calculate_hash(file_path)
            
            if file_hash in hashes:
                print(f"Duplicate found: {file_path} (same as {hashes[file_hash]})")
                os.remove(file_path)  # Remove the duplicate file
            else:
                hashes[file_hash] = file_path

for species in file_name:
    species_path = os.path.join('data/', species)
    remove_duplicate_images(species_path)
   
        


Duplicate found: data/chinese-mountain-cat\ch (20).jpg (same as data/chinese-mountain-cat\ch (10).jpg)
Duplicate found: data/chinese-mountain-cat\.ipynb_checkpoints\ch (11)-checkpoint.jpg (same as data/chinese-mountain-cat\ch (11).jpg)
Duplicate found: data/chinese-mountain-cat\.ipynb_checkpoints\ch (19)-checkpoint.jpg (same as data/chinese-mountain-cat\ch (19).jpg)
Duplicate found: data/european-wildcat\eu (3).jpg (same as data/european-wildcat\eu (26).jpg)
Duplicate found: data/european-wildcat\eu (5).jpg (same as data/european-wildcat\eu (27).jpg)
Duplicate found: data/european-wildcat\eu (62).jpg (same as data/european-wildcat\eu (13).jpg)
Duplicate found: data/european-wildcat\eu (64).jpg (same as data/european-wildcat\eu (15).jpg)
Duplicate found: data/european-wildcat\eu (69).jpg (same as data/european-wildcat\eu (17).jpg)
Duplicate found: data/european-wildcat\eu (7).jpg (same as data/european-wildcat\eu (32).jpg)
Duplicate found: data/european-wildcat\eu (73).jpg (same as data

### Problem 3. Loading a model (2 points)
Find a suitable, trained convolutional neural network classifier. I recommend `ResNet50` as it's small enough to run well on any machine and powerful enough to make reasonable predictions. Most ready-made classifiers have been trained for 1000 classes.

You'll need to install libraries and possibly tinker with configurations for this task. When you're done, display the total number of layers and the total number of parameters. For ResNet50, you should expect around 50 layers and 25M parameters.

### Problem 4. Prepare the images (1 point)
You'll need to prepare the images for passing to the model. To do so, they have to be resized to the same dimensions. Most available models have a specific requirement for sizes. You may need to do additional preprocessing, depending on the model requirements. These requirements should be easily available in the model documentation.

### Problem 5. Load the images efficiently (1 point)
Now that you've seen how to prepare the images for passing to the model... find a way to do it efficiently. Instead of loading the entire dataset in the RAM, read the images in batches (e.g. 4 images at a time). The goal is to read these, preprocess them, maybe save the preprocessed results in RAM.

If you've already done this in one of the previous problems, just skip this one. You'll get your point for it.

\* Even better, save the preprocessed image arrays (they will not be valid .jpg file) as separate files, so you can load them "lazily" in the following steps. This is a very common optimization to work with large datasets.

### Problem 6. Predictions (1 point)
Finally, you're ready to get into the meat of the problem. Obtain predictions from your model and evaluate them. This will likely involve manual work to decide how the returned classes relate to the original ones.

Create a [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) to evaluate the classification.

### Problem 7. Grayscale (1 point)
Converting the images to grayscale should affect the classification negatively, as we lose some of the color information.

Find a way to preprocess the images to grayscale (using what you already have in Problem 4 and 5), pass them to the model, and compare the classification results to the previous ones.

### Problem 8. Deep image features (1 point)
Find a way to extract one-dimensional vectors (features) for each (non-grayscale) image, using your model. This is typically done by "short-circuiting" the model output to be an intermediate layer, while keeping the input the same. 

In case the outputs (also called feature maps) have different shapes, you can flatten them in different ways. Try to not create huge vectors; the goal is to have a relatively short sequence of numbers which describes each image.

You may find a tutorial like [this](https://towardsdatascience.com/exploring-feature-extraction-with-cnns-345125cefc9a) pretty useful but note your implementation will depend on what model (and framework) you've decided to use.

It's a good idea to save these as one or more files, so you'll spare yourself a ton of preprocessing.

### Problem 9. Putting deep image features to use (1 points)
Try to find similar images, using a similarity metric on the features you got in the previous problem. Two good metrics are `mean squared error` and `cosine similarity`. How do they work? Can you spot images that look too similar? Can you explain why?

\* If we were to take Fourier features (in a similar manner, these should be a vector of about the same length), how do they compare to the deep features; i.e., which features are better to "catch" similar images?

### * Problem 10. Explore, predict, and evaluate further
You can do a ton of things here, at your desire. For example, how does masking different areas of the image affect classification - a method known as **saliency map** ([info](https://en.wikipedia.org/wiki/Saliency_map))? Can we detect objects? Can we significantly reduce the number of features (keeping the quality) that we get? Can we reliably train a model to predict our own classes? We'll look into these in detail in the future.